# Cubic Equations of State

These type of equations of state revolve around finding the compressibility factor $Z$ to correct the ideal gas law

$$
\rho = \frac{p}{RTZ},
$$

and obtain the density (and other thermodynamic properties) of a real fluid.

Cubic equations of state model the compressibility factor as roots of a normalized cubic polynomial

$$
Z^3 + c_2(A,B) Z^2 + c_1(A,B) Z + c_0(A, B) = 0,
$$

where the coefficients depend on dimensionless cohesion $A$ and covolume $B$.

PorePy provides functionality for finding the real roots of real cubic polynomials, and provide derivatives of the roots with respect to the coefficients and consequently $A,B$ and their dependencies on pressure and temperature.

This tutorial demonstrates the functionality and provides an example for a cubic EoS, the Peng-Robinson equation of state.
It also introduces compiled EoS computations and explains the rational behind the API.

### Table of contents

1. [Solving real cubic polynomials](#solving-real-cubic-polynomials)
2. [Compiled Equations of State](#compiled-equations-of-state)
3. [Example: Peng-Robinson EoS](#example-peng-robinson-eos)

## Solving real cubic polynomials

The roots are obtained dependent on the coefficients $c_2, c_1, c_0$.
By calculating the discriminant, we can determine the number of real roots and their algebraic multiplicity.
One of the following 4 cases can happen:

1. Single real root with multiplicity three.
1. Single real root and two complex-conjugated roots.
3. Two real roots where 1 has multiplicity three.
4. Three distinct real roots.

PorePy's approach is determining the root case and applying the correct trigonometric formula to obtain the real root.

In [2]:
import porepy.compositional.peng_robinson.cubic_polynomial as cp

c2 = 1.
c1 = -10
c0 = 0.5

eps = 1e-14  # For determining degenerate discriminant

r = cp.calculate_roots(c2, c1, c0, eps)

case = cp.get_root_case(c2, c1, c0, eps)

map_to_base_func = {
    0: cp.triple_root,
    1: cp.two_roots,
    2: cp.one_root,
    3: cp.three_roots,
}

if case in [0, 1]:
    assert r.size == 1
else:
    assert r.size == case

print(r)
print(map_to_base_func[case](c2, c1, c0))

[-3.72247101  0.05026536  2.67220564]
[-3.72247101  0.05026536  2.67220564]


As evident, the general function `calculate_roots` provides the same result as the base function.
In every roots case, the returned roots are ordered by size, starting with the smalles

Another featuer is that we can calculate the derivatives of the roots with respect to the coefficients.
We obtain a 2D array, which is assentialy the Jacobian of $r(c_2, c_1, c_0)$.
Note however that the Jacobian changes size. The number of rows always corresponds to the number of roots.

In [8]:
print(cp.calculate_root_derivatives(c2, c1, c0, eps))
print('---')
print(cp.calculate_roots(0, 1, 1, eps))
print(cp.calculate_root_derivatives(0, 1, 1, eps))

[[-5.74364514e-01  1.54296572e-01 -4.14500399e-02]
 [ 2.55422030e-04  5.08147214e-03  1.01092921e-01]
 [-4.25890908e-01 -1.59378044e-01 -5.96428812e-02]]
---
[-0.6823278]
[[-0.194254    0.28469308 -0.41723799]]


## Compiled Equations of State

## Example: Peng-Robinson EoS